# Recommender system using Pyspark (ALS algorithm)
---


#### Requirements:
- Python
- Apache Spark

## Import Libraries and Initialize spark session

In [1]:
# import libraries
from pyspark import SparkContext
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession, Row, SQLContext
from pyspark.sql.functions import explode, count, mean, split, col, to_timestamp, udf
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, Row, SQLContext

In [3]:
appName = "Analysis Application with PySpark"

#initialize the spark session
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2048m").\
        getOrCreate()

#get sparkcontext from the sparksession
sc = spark.sparkContext
sqlContext = SQLContext(sc)

24/03/20 08:13:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Load Dataset in Apache Spark


In [4]:
df_business = sqlContext.read.json('dataset/yelp_academic_dataset_business.json')

24/03/20 08:14:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df_review = sqlContext.read.json('dataset/yelp_academic_dataset_review.json')

In [ ]:
df_user = sqlContext.read.json('dataset/yelp_academic_dataset_user.json')

# Exploratory Data Analysis (EDA)

## Business Dataset

In [6]:
df_business.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [7]:
df_business.show(5)

+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|         city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|[,,,,,,,,,,, True...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...|Santa Barbara|                null|      0|34.4266787|-119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|[,,,,,,,,, True,,...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|       Affton|[8:0-18:30, 0:0-0...|      1| 38.551126|  -90.335695|    

In [8]:
df_business.describe().show()

+-------+-----------------+--------------------+--------------------+-----------+------------------+-----------------+------------------+--------------------+------------------+------------------+------------------+------+
|summary|          address|         business_id|          categories|       city|           is_open|         latitude|         longitude|                name|       postal_code|      review_count|             stars| state|
+-------+-----------------+--------------------+--------------------+-----------+------------------+-----------------+------------------+--------------------+------------------+------------------+------------------+------+
|  count|           150346|              150346|              150243|     150346|            150346|           150346|            150346|              150346|            150346|            150346|            150346|150346|
|   mean|7369.333333333333|                null|                null|       null|0.7961502135075094|36.67115

In [9]:
# Total number of businesses
df_business.count()

150346

In [10]:
# Actual open businesses
category_counts = df_business.groupBy("is_open").count().show()

+-------+------+
|is_open| count|
+-------+------+
|      0| 30648|
|      1|119698|
+-------+------+



In [11]:
average_stars = df_business.agg({"stars": "avg"}).collect()[0][0]
print(f"Average stars: {average_stars}")

Average stars: 3.5967235576603303


In [12]:
# Stars distribution
df_business.groupBy("stars").count().sort("stars").show()

+-----+-----+
|stars|count|
+-----+-----+
|  1.0| 1986|
|  1.5| 4932|
|  2.0| 9527|
|  2.5|14316|
|  3.0|18453|
|  3.5|26519|
|  4.0|31125|
|  4.5|27181|
|  5.0|16307|
+-----+-----+



In [13]:
df_business.select("categories").show(truncate=False)

+----------------------------------------------------------------------------------------------------------+
|categories                                                                                                |
+----------------------------------------------------------------------------------------------------------+
|Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists|
|Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services                            |
|Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores                        |
|Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries                                                     |
|Brewpubs, Breweries, Food                                                                                 |
|Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants                              |
|Sporting Goods, Fa

In [14]:
# Analysis over attributes
attribute_columns = df_business.select("Attributes.*").columns

In [15]:
for column in attribute_columns:
  print("Distribution for column: Attribute." + column)
  df_business.groupBy("Attributes." + column).count().show(truncate=False)

Distribution for column: Attribute.AcceptsInsurance


+----------------+------+
|AcceptsInsurance|count |
+----------------+------+
|None            |8     |
|False           |1767  |
|null            |144633|
|True            |3938  |
+----------------+------+

Distribution for column: Attribute.AgesAllowed


+-----------+------+
|AgesAllowed|count |
+-----------+------+
|null       |150217|
|u'18plus'  |6     |
|u'21plus'  |46    |
|u'allages' |77    |
+-----------+------+

Distribution for column: Attribute.Alcohol


+----------------+------+
|Alcohol         |count |
+----------------+------+
|None            |38    |
|u'none'         |15977 |
|null            |107157|
|'beer_and_wine' |1369  |
|'none'          |4933  |
|u'beer_and_wine'|4880  |
|u'full_bar'     |12968 |
|'full_bar'      |3024  |
+----------------+------+

Distribution for column: Attribute.Ambience


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Ambience                                                                                                                                                              |count|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'casual': True}                         |14   |
|{u'divey': False, u'hipster': False, u'casual': True, u'touristy': None, u'trendy': None, u'intimate': None, u'romantic': None, u'classy': None, u'upscale': False}   |8    |
|{u'divey': None, u'hipster': True, u'casual': True, u'touristy': None, u'trendy': True, u'intimate': False, u'romantic': Fal

...

+-----------------------+------+
|RestaurantsTableService|count |
+-----------------------+------+
|None                   |15    |
|False                  |7293  |
|null                   |130364|
|True                   |12674 |
+-----------------------+------+

Distribution for column: Attribute.RestaurantsTakeOut


+------------------+-----+
|RestaurantsTakeOut|count|
+------------------+-----+
|None              |2105 |
|False             |4809 |
|null              |90489|
|True              |52943|
+------------------+-----+

Distribution for column: Attribute.Smoking


+----------+------+
|Smoking   |count |
+----------+------+
|u'outdoor'|1800  |
|None      |14    |
|null      |145779|
|'no'      |30    |
|u'no'     |2375  |
|'outdoor' |17    |
|u'yes'    |331   |
+----------+------+

Distribution for column: Attribute.WheelchairAccessible


+--------------------+------+
|WheelchairAccessible|count |
+--------------------+------+
|None                |27    |
|False               |2933  |
|null                |121393|
|True                |25993 |
+--------------------+------+

Distribution for column: Attribute.WiFi


+-------+-----+
|WiFi   |count|
+-------+-----+
|None   |50   |
|'paid' |133  |
|null   |93432|
|'no'   |6610 |
|u'paid'|486  |
|u'no'  |15221|
|u'free'|27029|
|'free' |7385 |
+-------+-----+



## Reviews Dataset

In [16]:
df_review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [17]:
df_review.show(5)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [18]:
df_review.count()

6990280

In [19]:
# Numerical columns, since they have similar structure and distribution,
# the analysis will be the same
for column in ["cool", "funny", "useful"]:
  print("\n\nAnalysis for column: " + column)
  df_review.select(column).describe().show()
  df_review.filter(col(column) < 0).show()
  print("Number of -1 rows for the column " + column + " are: " + str(df_review.filter(col(column) < 0).count()))



Analysis for column: cool


+-------+------------------+
|summary|              cool|
+-------+------------------+
|  count|           6990280|
|   mean|0.4986175088837643|
| stddev| 2.172459820211187|
|    min|                -1|
|    max|               404|
+-------+------------------+



+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|E3Qk7xTznkT0SABL2...|  -1|2017-10-05 02:47:33|   -1|JcWLycg0Hdsz6LBNA...|  5.0|The food was so m...|    -1|04j3r2Psfw_LxX1UX...|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+



Number of -1 rows for the column cool are: 1


Analysis for column: funny


+-------+-------------------+
|summary|              funny|
+-------+-------------------+
|  count|            6990280|
|   mean|0.32655959417934616|
| stddev| 1.6887290985540495|
|    min|                 -1|
|    max|                792|
+-------+-------------------+



+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|E3Qk7xTznkT0SABL2...|  -1|2017-10-05 02:47:33|   -1|JcWLycg0Hdsz6LBNA...|  5.0|The food was so m...|    -1|04j3r2Psfw_LxX1UX...|
|qjstXewt23oCZnf86...|   2|2017-02-25 23:40:34|   -1|7tlF2NEDiFqMRjKBg...|  1.0|The most horrible...|     2|VKK7pPl6DfjL9H5b3...|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+



Number of -1 rows for the column funny are: 2


Analysis for column: useful


+-------+------------------+
|summary|            useful|
+-------+------------------+
|  count|           6990280|
|   mean|1.1846089140921394|
| stddev| 3.253766966933363|
|    min|                -1|
|    max|              1182|
+-------+------------------+



+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|SJQePJL1bkeEk-6gp...|   0|2018-12-23 01:35:31|    0|plhadSWfXZFKRAhEz...|  1.0|This complaint ha...|    -1|0Q4ouqkcwrVE_Im6a...|
|SJQePJL1bkeEk-6gp...|   0|2016-01-17 00:21:53|    0|gk_9N_c-7oEgzrXVD...|  1.0|Only bounce play ...|    -1|lEOLrNxD4wknYyERp...|
|SJQePJL1bkeEk-6gp...|   0|2017-01-18 18:11:15|    0|iYS4pfUCUPT_TEmW6...|  3.0|Having been to bo...|    -1|1p8Pu1pvAsuNtuZDt...|
|E3Qk7xTznkT0SABL2...|  -1|2017-10-05 02:47:33|   -1|JcWLycg0Hdsz6LBNA...|  5.0|The food was so m...|    -1|04j3r2Psfw_LxX1UX...|
+--------------------+----+-------------------+-----+--------------------+-----+----------

Number of -1 rows for the column useful are: 4


In [20]:
df_review.groupBy("stars").count().sort("stars").show()

+-----+-------+
|stars|  count|
+-----+-------+
|  1.0|1069561|
|  2.0| 544240|
|  3.0| 691934|
|  4.0|1452918|
|  5.0|3231627|
+-----+-------+



In [21]:
# Look for null values in foreign keys and empty reviews
for column in ["business_id", "user_id", "text"]:
  print("\n\nAnalysis for column: " + column)
  print("Number of NULL " + column + " rows: " + str(df_review.filter(col(column).isNull()).count()))
  print("Number of EMPTY ('') " + column + " rows: " + str(df_review.filter(col(column) == "").count()))



Analysis for column: business_id


Number of NULL business_id rows: 0


Number of EMPTY ('') business_id rows: 0


Analysis for column: user_id


Number of NULL user_id rows: 0


Number of EMPTY ('') user_id rows: 0


Analysis for column: text


Number of NULL text rows: 0


Number of EMPTY ('') text rows: 0


## User Dataset

In [ ]:
df_user.printSchema()

In [ ]:
df_user.show(5)

In [ ]:
df_user.describe().show()

In [ ]:
df_user.count()

# Feature Engineering
Modify the datasets based on the analysis made

In [ ]:
# Join all 3 datasets

## Business Dataset

In [ ]:
# Discard useless attributes (that has more null values than actual usuful ones)

In [ ]:
# Uniform attributes (transforming them in Boolean and setting NULL = None = False)

In [23]:
# Transform "categories" column to split elements
df_business = df_business.withColumn("categories_array", split("categories", ", "))
df_business.select("categories", "categories_array").show(truncate=False)

+----------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------+
|categories                                                                                                |categories_array                                                                                            |
+----------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------+
|Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists|[Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists]|
|Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services                            |[Shipping Centers, L

In [24]:
categories_distribution = df_business.select("business_id", explode("categories_array").alias("category")).groupBy("category").agg(count("business_id").alias("count"))

In [25]:
categories_distribution.sort("count", ascending=False).show(50, truncate=False)
print(f"Number of unique categories: {categories_distribution.count()}")
mean = categories_distribution.agg(mean("count")).collect()[0][0]
print(f"Average occurences of a category: {mean}")

+-------------------------+-----+
|category                 |count|
+-------------------------+-----+
|Restaurants              |52268|
|Food                     |27781|
|Shopping                 |24395|
|Home Services            |14356|
|Beauty & Spas            |14292|
|Nightlife                |12281|
|Health & Medical         |11890|
|Local Services           |11198|
|Bars                     |11065|
|Automotive               |10773|
|Event Planning & Services|9895 |
|Sandwiches               |8366 |
|American (Traditional)   |8139 |
|Active Life              |7687 |
|Pizza                    |7093 |
|Coffee & Tea             |6703 |
|Fast Food                |6472 |
|Breakfast & Brunch       |6239 |
|American (New)           |6097 |
|Hotels & Travel          |5857 |
|Home & Garden            |5799 |
|Fashion                  |5739 |
|Burgers                  |5636 |
|Arts & Entertainment     |5434 |
|Auto Repair              |5433 |
|Hair Salons              |5046 |
|Nail Salons  

Number of unique categories: 1311


Average occurences of a category: 509.9862700228833


## Review Dataset

In [26]:
# Convert "date" column to date format
df_review = df_review.withColumn("date", to_timestamp(df_review.date, "yyyy-MM-dd HH:mm:ss"))

## User Dataset

# Sentiment Analysis

## Using trained model with stars

In [27]:
from pyspark.ml.feature import HashingTF, IDF, RegexTokenizer, StopWordsRemover, StringIndexer, IndexToString
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
def stars_transformer(stars):
  if stars >= 4:
    return "positive"
  if stars == 3:
    return "neutral"
  else:
    return "negative"

stars_transf_udf = udf(lambda x: stars_transformer(x), StringType())

# Pass from stars to sentiment
df_review_sa = df_review.withColumn("sentiment", stars_transf_udf(col("stars")))

In [29]:
df_review_sa.groupBy("sentiment").count().orderBy(col("count").desc()).show()

+---------+-------+
|sentiment|  count|
+---------+-------+
| positive|4684545|
| negative|1613801|
|  neutral| 691934|
+---------+-------+



In [30]:
# Calculate class weights to balance the dataset
class_weights = df_review_sa.groupBy("sentiment").count()
total_count = df_review_sa.count()
class_weights = class_weights.withColumn("classWeights", total_count / (class_weights["count"] * class_weights.count()))

In [31]:
# RegexTokenizer for more advanced tokenization
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

# StopWordsRemover to remove common words
add_stopwords = ["http","https","amp","rt","t","c","the"]
stop_words_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words").setStopWords(add_stopwords)

# Apply TF-IDF
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# StringIndexer for converting text labels to numerical values
label_indexer = StringIndexer(inputCol="sentiment", outputCol="label", handleInvalid="skip")

# # Create a Logistic Regression model
# lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# # Create a Decision Tree classifier
# dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=5)

# Create a Gradient Boost classifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, maxDepth=5, seed=42, weightCol="classWeights")

# Train the StringIndexer on the entire dataset to obtain the labels
label_model = label_indexer.fit(df_review_sa)
labels = label_model.labels

# IndexToString to convert predicted labels back to text
label_converter = IndexToString(inputCol="prediction", outputCol="prediction_text", labels=labels)

In [32]:
# Create a pipeline with all stages
pipeline = Pipeline(stages=[tokenizer,
                            stop_words_remover,
                            hashingTF,
                            idf,
                            label_indexer,
                            rf,
                            label_converter])

In [33]:
# Split the dataset
(trainingData, testData) = df_review_sa.randomSplit([0.8, 0.2], seed=42)

In [34]:
# Adding weights to the training dataset to train the model
trainingData_weighted = trainingData.join(class_weights, "sentiment")

In [35]:
# Train the pipeline
model = pipeline.fit(trainingData_weighted)

24/03/20 08:52:40 WARN HeartbeatReceiver: Removing executor 1 with no recent heartbeats: 139422 ms exceeds timeout 120000 ms
24/03/20 08:52:41 ERROR TaskSchedulerImpl: Lost executor 1 on 172.25.0.4: Executor heartbeat timed out after 139422 ms
24/03/20 08:52:41 WARN TaskSetManager: Lost task 70.0 in stage 461.0 (TID 10083, 172.25.0.4, executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 139422 ms
24/03/20 08:52:52 WARN TaskSetManager: Lost task 70.1 in stage 461.0 (TID 10282, 172.25.0.4, executor 2): FetchFailed(null, shuffleId=76, mapIndex=-1, mapId=-1, reduceId=70, message=
org.apache.spark.shuffle.MetadataFetchFailedException: Missing an output location for shuffle 76
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$2(MapOutputTracker.scala:1010)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$2$adapted(MapOutputTracker.scala:1006)
	at scala.collection.Iterator.foreach(

KeyboardInterrupt: 

In [ ]:
# Save the model
model.write().overwrite().save("models/cluster_models/sentiment_analysis_rf")

In [ ]:
model = PipelineModel.load("models/cluster_models/sentiment_analysis_rf")

In [ ]:
# Make predictions on the test data
predictions = model.transform(testData)

In [ ]:
predictions.groupBy("prediction_text").count().show()

In [ ]:
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1_score}")

## Using HuggingFace

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

sentiment_analysis = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", max_length=512, truncation=True)

In [ ]:
@F.pandas_udf('string')
def sentiment_udf(texts: pd.Series) -> pd.Series:
  translations = [result['label'] for result in sentiment_analysis(texts.to_list())]
  return pd.Series(translations)

In [ ]:
predictions.show()

In [ ]:
predictions = predictions.withColumn("bert-sentiment", sentiment_udf(predictions.text))

In [ ]:
predictions.show()

# Recommandation System with ALS algorithm

# Code already written in the notebook (old)

## Filter Rows and columns



In [ ]:
df_business = df_business.select("business_id","name", "stars",
                                 "review_count", "attributes",
                                 "categories", "city").withColumnRenamed("stars", "stars_restaurant")

df_business = df_business.filter((df_business['city'] == 'Toronto') & (df_business.categories.contains('Restaurants'))).drop('city')

In [ ]:
df_review = df_review.join(df_business, on='business_id', how='inner')

Lets make a quick visualisation to the basic elements of our review table.

In [ ]:
df_review.select(['business_id', 'user_id', 'stars']).show()

## Exploratory Data Analysis



In [ ]:
reviews = df_review.select('stars').collect()
review_list = [reviews[i][0] for i in range(len(reviews))]

plt.hist(review_list, bins=[0.5,1.5,2.5,3.5,4.5,5.5], alpha=0.5,
         histtype='stepfilled', color='steelblue',
         edgecolor='none')
plt.ylabel('Frequency')
plt.xlabel('Rating')
plt.style.use('seaborn-white')

Quite generous public from Toronto. Most ratings are above 3. Now lets see the the distrubtion of ratings of each restaurants.

In [ ]:
restaurant_reviews = df_business.select('stars_restaurant').collect()
restaurant_reviews_list = [restaurant_reviews[i][0] for i in range(len(restaurant_reviews))]


plt.hist(restaurant_reviews_list, bins=[0.5,1.5,2.5,3.5,4.5,5.5], alpha=0.5,
         histtype='stepfilled', color='steelblue',
         edgecolor='none')
plt.ylabel('Frequency')
plt.xlabel('Rating')
plt.style.use('seaborn-white')

Here were see a more normally distributed curve. Nevertheless most restaurants do pretty well.
Now lets visualize what are the most popular type of restaurants in Toronto. What kind of food do they serve? We will create a wordcloud.

In [ ]:
restaurant_categories = df_business.select('categories').collect()
restaurant_categories_list = [restaurant_categories[i][0] for i in range(len(restaurant_categories))]

In [ ]:
text = " ".join(review for review in restaurant_categories_list)

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# eliminate useless words
text = text.replace('Restaurants', "")
text = text.replace('bars', "")
text = text.replace('food', "")


# Generate a word cloud image
wordcloud = WordCloud(background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.figure(figsize=(10,8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

> ## Convert String to index


In [ ]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in ['business_id', 'user_id']]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(df_review).transform(df_review)
transformed.select(['business_id', 'user_id','business_id_index', 'user_id_index'])

## Split Dataset in train and test


In [ ]:
(training, test) = transformed.randomSplit([0.8, 0.2])

## Create ALS model


In [ ]:
als=ALS(maxIter=5,
        regParam=0.09,
        rank=25,
        userCol="user_id_index",
        itemCol="business_id_index",
        ratingCol="stars",
        coldStartStrategy="drop",
        nonnegative=True)

model=als.fit(training)

## Evaluate RMSE

In [ ]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="stars",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))

## Visualize Recommendations



In [ ]:
test = model.recommendForAllUsers(20).filter(col('user_id_index')==30).select("recommendations").collect()
topRestaurants = []
for item in test[0][0]:
    topRestaurants.append(item.business_id_index)

schema = StructType([StructField("business_id_index",IntegerType(),True)])
restaurants = spark.createDataFrame(topRestaurants,IntegerType()).toDF("business_id_index")


transformed\
.select(['business_id', 'user_id', 'stars', 'categories'])\
.filter(col('user_id_index')==30)\
.show()

restaurants\
.join(transformed, on = 'business_id_index', how = 'inner')\
.select(['business_id', 'stars', 'categories', 'name'])\
.drop_duplicates(subset=['name'])\
.show()

